# **Código para pruebas de estabilidad de métricas**

Compara el desempeño de la media y la mediana contra la métrica desarrollada en *defs.py* & *defs2.py*, así como contra sí mismas para un diagnóstico de mejoras de *defs.py*, mediante una simulación de 10,000 distribuciones diferentes, generadas aleatoriamente (función de generación aleatoria ubicada en **funciones_aux.py**)

In [ ]:
# Bloque 0: imports y setup
import numpy as np
import pandas as pd
from tqdm import tqdm
import defs
import defs2
import funciones_aux
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon
from funciones_aux import (
    eval_one_sim, bowley_skew, excess_kurtosis, 
    run_resumenes, run_tests, victorias_grouped, 
    columnas_comparacion_default
)

RANDOM_SEED = 123
rng = np.random.default_rng(RANDOM_SEED)

In [ ]:
# Bloque 3: loop principal de simulación (1000)

S = 10000
rows = []
for s in tqdm(range(1, S+1)):
    rows.append(eval_one_sim(s, rng))

df_ab = pd.DataFrame(rows)

print("✅ Simulaciones completadas:", len(df_ab))
print("Columnas:", list(df_ab.columns))
df_ab.head()

# Bloque 4: Resumen de victorias para recorte unilateral
comparisons_left = [
    ("d_mp1_vs_mean_left",   "V1 vs Media (izq)"),
    ("d_mp2_vs_mean_left",   "V2 vs Media (izq)"),
    ("d_mp1_vs_median_left", "V1 vs Mediana (izq)"),
    ("d_mp2_vs_median_left", "V2 vs Mediana (izq)"),
    ("d_v2_vs_v1_left",      "V2 vs V1 (izq)")
]
comparisons_right = [
    ("d_mp1_vs_mean_right",   "V1 vs Media (der)"),
    ("d_mp2_vs_mean_right",   "V2 vs Media (der)"),
    ("d_mp1_vs_median_right", "V1 vs Mediana (der)"),
    ("d_mp2_vs_median_right", "V2 vs Mediana (der)"),
    ("d_v2_vs_v1_right",      "V2 vs V1 (der)")
]

# Resumen global de victorias (proporciones y diferencias promedio)
df_resumenes_left = run_resumenes(df_ab, comparisons_left, threshold=0.0)
df_resumenes_right = run_resumenes(df_ab, comparisons_right, threshold=0.0)
print("--- Resumen de recorte izquierda ---")
display(df_resumenes_left)
print("\n--- Resumen de recorte derecha ---")
display(df_resumenes_right)


In [ ]:
# Bloque 4: Resumen de victorias por subgrupos para recorte unilateral
cols_diffs_left = ["d_mp1_vs_mean_left", "d_mp2_vs_mean_left", "d_v2_vs_v1_left"]
cols_diffs_right = ["d_mp1_vs_mean_right", "d_mp2_vs_mean_right", "d_v2_vs_v1_right"]

victorias_familia_left   = victorias_grouped(df_ab, "familia",       cols_diffs_left, threshold=0.0)
victorias_familia_right  = victorias_grouped(df_ab, "familia",       cols_diffs_right, threshold=0.0)
victorias_outliers_left  = victorias_grouped(df_ab, "tuvo_outliers", cols_diffs_left, threshold=0.0)
victorias_outliers_right = victorias_grouped(df_ab, "tuvo_outliers", cols_diffs_right, threshold=0.0)
victorias_rango_n_left   = victorias_grouped(df_ab, "rango_n",       cols_diffs_left, threshold=0.0)
victorias_rango_n_right  = victorias_grouped(df_ab, "rango_n",       cols_diffs_right, threshold=0.0)

print("--- Proporción de victorias por familia (izquierda) ---")
display(victorias_familia_left)
print("\n--- Proporción de victorias por familia (derecha) ---")
display(victorias_familia_right)

print("\n--- Proporción de victorias por outliers (izquierda) ---")
display(victorias_outliers_left)
print("\n--- Proporción de victorias por outliers (derecha) ---")
display(victorias_outliers_right)

print("\n--- Proporción de victorias por rango_n (izquierda) ---")
display(victorias_rango_n_left)
print("\n--- Proporción de victorias por rango_n (derecha) ---")
display(victorias_rango_n_right)


In [ ]:
# Tests estadísticos (Wilcoxon unilateral) para recorte unilateral

# Definir las columnas de diferencia V1 vs V2 para ambos recortes
df_ab["d_v1_vs_v2_left"] = df_ab["delta_mp1_left"] - df_ab["delta_mp2_left"]
df_ab["d_v1_vs_v2_right"] = df_ab["delta_mp1_right"] - df_ab["delta_mp2_right"]

comparisons_left = [
    ("d_mp1_vs_mean_left",   "V1 vs Media (izq)"),
    ("d_mp2_vs_mean_left",   "V2 vs Media (izq)"),
    ("d_mp1_vs_median_left", "V1 vs Mediana (izq)"),
    ("d_mp2_vs_median_left", "V2 vs Mediana (izq)"),
    ("d_v2_vs_v1_left",      "V2 vs V1 (izq)")
]

comparisons_right = [
    ("d_mp1_vs_mean_right",   "V1 vs Media (der)"),
    ("d_mp2_vs_mean_right",   "V2 vs Media (der)"),
    ("d_mp1_vs_median_right", "V1 vs Mediana (der)"),
    ("d_mp2_vs_median_right", "V2 vs Mediana (der)"),
    ("d_v2_vs_v1_right",      "V2 vs V1 (der)")
]

# Ejecutar los tests existentes
df_tests_left = run_tests(df_ab, comparisons_left, alternative="greater")
df_tests_right = run_tests(df_ab, comparisons_right, alternative="greater")

# Ejecutar la nueva prueba V1 vs V2 (Wilcoxon unilateral, alternativa 'greater')
comparisons_left_v1_vs_v2 = [("d_v1_vs_v2_left", "V1 vs V2 (izq)")]
comparisons_right_v1_vs_v2 = [("d_v1_vs_v2_right", "V1 vs V2 (der)")]

df_tests_left_v1_vs_v2 = run_tests(df_ab, comparisons_left_v1_vs_v2, alternative="greater")
df_tests_right_v1_vs_v2 = run_tests(df_ab, comparisons_right_v1_vs_v2, alternative="greater")

print("--- Tests Wilcoxon recorte izquierda ---")
display(df_tests_left)
print("\n--- Tests Wilcoxon recorte derecha ---")
display(df_tests_right)

print("\n--- Test Wilcoxon V1 vs V2 (izquierda) ---")
display(df_tests_left_v1_vs_v2)

print("\n--- Test Wilcoxon V1 vs V2 (derecha) ---")
display(df_tests_right_v1_vs_v2)

In [ ]:
# Exportar los nuevos DataFrames de resultados y victorias

df_resumenes_left.to_csv("ab_resumenes_left.csv", index=False)
df_resumenes_right.to_csv("ab_resumenes_right.csv", index=False)
victorias_familia_left.to_csv("ab_victorias_por_familia_left.csv", index=False)
victorias_familia_right.to_csv("ab_victorias_por_familia_right.csv", index=False)
victorias_outliers_left.to_csv("ab_victorias_por_outliers_left.csv", index=False)
victorias_outliers_right.to_csv("ab_victorias_por_outliers_right.csv", index=False)
victorias_rango_n_left.to_csv("ab_victorias_por_rangon_left.csv", index=False)
victorias_rango_n_right.to_csv("ab_victorias_por_rangon_right.csv", index=False)
df_tests_left.to_csv("ab_tests_wilcoxon_left.csv", index=False)
df_tests_right.to_csv("ab_tests_wilcoxon_right.csv", index=False)
print("CSV exportados ✅")


### **Visualizaciones**

In [ ]:
# --- 1. Dispersión: delta_mean vs delta_mp1 y delta_mp2, y delta_median vs delta_mp1 y delta_mp2 bajo recorte unilateral ---
fig, axes = plt.subplots(2, 4, figsize=(28, 12), sharex=True, sharey=True)

# Izquierda
sns.scatterplot(
    data=df_ab, x="delta_mean_left", y="delta_mp1_left",
    hue="familia", alpha=0.6, ax=axes[0,0], legend=False
)
axes[0,0].plot([0, df_ab[["delta_mean_left","delta_mp1_left"]].max().max()],
             [0, df_ab[["delta_mean_left","delta_mp1_left"]].max().max()],
             'r--')
axes[0,0].set_title("V1 vs Media (delta, recorte izquierda)")
axes[0,0].set_xlabel("Δ Media (izq)")
axes[0,0].set_ylabel("Δ Métrica V1 (izq)")

sns.scatterplot(
    data=df_ab, x="delta_mean_left", y="delta_mp2_left",
    hue="familia", alpha=0.6, ax=axes[0,1]
)
axes[0,1].plot([0, df_ab[["delta_mean_left","delta_mp2_left"]].max().max()],
             [0, df_ab[["delta_mean_left","delta_mp2_left"]].max().max()],
             'r--')
axes[0,1].set_title("V2 vs Media (delta, recorte izquierda)")
axes[0,1].set_xlabel("Δ Media (izq)")
axes[0,1].set_ylabel("Δ Métrica V2 (izq)")

sns.scatterplot(
    data=df_ab, x="delta_median_left", y="delta_mp1_left",
    hue="familia", alpha=0.6, ax=axes[0,2], legend=False
)
axes[0,2].plot([0, df_ab[["delta_median_left","delta_mp1_left"]].max().max()],
             [0, df_ab[["delta_median_left","delta_mp1_left"]].max().max()],
             'r--')
axes[0,2].set_title("V1 vs Mediana (delta, recorte izquierda)")
axes[0,2].set_xlabel("Δ Mediana (izq)")
axes[0,2].set_ylabel("Δ Métrica V1 (izq)")

sns.scatterplot(
    data=df_ab, x="delta_median_left", y="delta_mp2_left",
    hue="familia", alpha=0.6, ax=axes[0,3]
)
axes[0,3].plot([0, df_ab[["delta_median_left","delta_mp2_left"]].max().max()],
             [0, df_ab[["delta_median_left","delta_mp2_left"]].max().max()],
             'r--')
axes[0,3].set_title("V2 vs Mediana (delta, recorte izquierda)")
axes[0,3].set_xlabel("Δ Mediana (izq)")
axes[0,3].set_ylabel("Δ Métrica V2 (izq)")

# Derecha
sns.scatterplot(
    data=df_ab, x="delta_mean_right", y="delta_mp1_right",
    hue="familia", alpha=0.6, ax=axes[1,0], legend=False
)
axes[1,0].plot([0, df_ab[["delta_mean_right","delta_mp1_right"]].max().max()],
             [0, df_ab[["delta_mean_right","delta_mp1_right"]].max().max()],
             'r--')
axes[1,0].set_title("V1 vs Media (delta, recorte derecha)")
axes[1,0].set_xlabel("Δ Media (der)")
axes[1,0].set_ylabel("Δ Métrica V1 (der)")

sns.scatterplot(
    data=df_ab, x="delta_mean_right", y="delta_mp2_right",
    hue="familia", alpha=0.6, ax=axes[1,1]
)
axes[1,1].plot([0, df_ab[["delta_mean_right","delta_mp2_right"]].max().max()],
             [0, df_ab[["delta_mean_right","delta_mp2_right"]].max().max()],
             'r--')
axes[1,1].set_title("V2 vs Media (delta, recorte derecha)")
axes[1,1].set_xlabel("Δ Media (der)")
axes[1,1].set_ylabel("Δ Métrica V2 (der)")

sns.scatterplot(
    data=df_ab, x="delta_median_right", y="delta_mp1_right",
    hue="familia", alpha=0.6, ax=axes[1,2], legend=False
)
axes[1,2].plot([0, df_ab[["delta_median_right","delta_mp1_right"]].max().max()],
             [0, df_ab[["delta_median_right","delta_mp1_right"]].max().max()],
             'r--')
axes[1,2].set_title("V1 vs Mediana (delta, recorte derecha)")
axes[1,2].set_xlabel("Δ Mediana (der)")
axes[1,2].set_ylabel("Δ Métrica V1 (der)")

sns.scatterplot(
    data=df_ab, x="delta_median_right", y="delta_mp2_right",
    hue="familia", alpha=0.6, ax=axes[1,3]
)
axes[1,3].plot([0, df_ab[["delta_median_right","delta_mp2_right"]].max().max()],
             [0, df_ab[["delta_median_right","delta_mp2_right"]].max().max()],
             'r--')
axes[1,3].set_title("V2 vs Mediana (delta, recorte derecha)")
axes[1,3].set_xlabel("Δ Mediana (der)")
axes[1,3].set_ylabel("Δ Métrica V2 (der)")

plt.tight_layout()
plt.show()


In [ ]:
# --- 2. Histograma de mejoras directas bajo recorte unilateral ---
fig, axes = plt.subplots(2, 5, figsize=(30, 10))

# Izquierda
sns.histplot(df_ab["d_mp1_vs_mean_left"], bins=40, kde=True, ax=axes[0,0], color="skyblue")
axes[0,0].axvline(0, color="red", linestyle="--")
axes[0,0].set_title("V1 – Media (Δ estabilidad, izq)")

sns.histplot(df_ab["d_mp2_vs_mean_left"], bins=40, kde=True, ax=axes[0,1], color="lightgreen")
axes[0,1].axvline(0, color="red", linestyle="--")
axes[0,1].set_title("V2 – Media (Δ estabilidad, izq)")

sns.histplot(df_ab["d_mp1_vs_median_left"], bins=40, kde=True, ax=axes[0,2], color="deepskyblue")
axes[0,2].axvline(0, color="red", linestyle="--")
axes[0,2].set_title("V1 – Mediana (Δ estabilidad, izq)")

sns.histplot(df_ab["d_mp2_vs_median_left"], bins=40, kde=True, ax=axes[0,3], color="limegreen")
axes[0,3].axvline(0, color="red", linestyle="--")
axes[0,3].set_title("V2 – Mediana (Δ estabilidad, izq)")

sns.histplot(df_ab["d_v2_vs_v1_left"], bins=40, kde=True, ax=axes[0,4], color="orange")
axes[0,4].axvline(0, color="red", linestyle="--")
axes[0,4].set_title("V2 – V1 (Δ estabilidad, izq)")

# Derecha
sns.histplot(df_ab["d_mp1_vs_mean_right"], bins=40, kde=True, ax=axes[1,0], color="skyblue")
axes[1,0].axvline(0, color="red", linestyle="--")
axes[1,0].set_title("V1 – Media (Δ estabilidad, der)")

sns.histplot(df_ab["d_mp2_vs_mean_right"], bins=40, kde=True, ax=axes[1,1], color="lightgreen")
axes[1,1].axvline(0, color="red", linestyle="--")
axes[1,1].set_title("V2 – Media (Δ estabilidad, der)")

sns.histplot(df_ab["d_mp1_vs_median_right"], bins=40, kde=True, ax=axes[1,2], color="deepskyblue")
axes[1,2].axvline(0, color="red", linestyle="--")
axes[1,2].set_title("V1 – Mediana (Δ estabilidad, der)")

sns.histplot(df_ab["d_mp2_vs_median_right"], bins=40, kde=True, ax=axes[1,3], color="limegreen")
axes[1,3].axvline(0, color="red", linestyle="--")
axes[1,3].set_title("V2 – Mediana (Δ estabilidad, der)")

sns.histplot(df_ab["d_v2_vs_v1_right"], bins=40, kde=True, ax=axes[1,4], color="orange")
axes[1,4].axvline(0, color="red", linestyle="--")
axes[1,4].set_title("V2 – V1 (Δ estabilidad, der)")

plt.tight_layout()
plt.show()


In [ ]:
# --- 3. Proporciones de victorias por familia bajo recorte unilateral ---
cols_diffs_left = ["d_mp1_vs_mean_left", "d_mp2_vs_mean_left", "d_mp1_vs_median_left", "d_mp2_vs_median_left", "d_v2_vs_v1_left"]
cols_diffs_right = ["d_mp1_vs_mean_right", "d_mp2_vs_mean_right", "d_mp1_vs_median_right", "d_mp2_vs_median_right", "d_v2_vs_v1_right"]

prop_fam_left = (
    df_ab.groupby("familia")[cols_diffs_left]
         .apply(lambda g: (g > 0).mean())
         .reset_index()
         .melt(id_vars="familia", var_name="comparacion", value_name="prop_victorias")
)
prop_fam_right = (
    df_ab.groupby("familia")[cols_diffs_right]
         .apply(lambda g: (g > 0).mean())
         .reset_index()
         .melt(id_vars="familia", var_name="comparacion", value_name="prop_victorias")
)

plt.figure(figsize=(16, 6))
sns.barplot(data=prop_fam_left, x="familia", y="prop_victorias", hue="comparacion")
plt.axhline(0.5, color="red", linestyle="--")
plt.title("Proporción de victorias por familia (recorte izquierda)")
plt.ylabel("Proporción de casos donde Δ > 0")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.legend(title="Comparación")
plt.show()

plt.figure(figsize=(16, 6))
sns.barplot(data=prop_fam_right, x="familia", y="prop_victorias", hue="comparacion")
plt.axhline(0.5, color="red", linestyle="--")
plt.title("Proporción de victorias por familia (recorte derecha)")
plt.ylabel("Proporción de casos donde Δ > 0")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.legend(title="Comparación")
plt.show()


In [ ]:
# --- 4. Heatmap de victorias por (familia x outliers) bajo recorte unilateral ---
heat_data_left_v1_median = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_mp1_vs_median_left"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)
heat_data_left_v2_median = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_mp2_vs_median_left"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)
heat_data_left_v2_v1 = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_v2_vs_v1_left"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)

heat_data_right_v1_median = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_mp1_vs_median_right"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)
heat_data_right_v2_median = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_mp2_vs_median_right"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)
heat_data_right_v2_v1 = (
    df_ab.groupby(["familia", "tuvo_outliers"])["d_v2_vs_v1_right"]
         .apply(lambda x: (x > 0).mean())
         .unstack()
)

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_left_v1_median, annot=True, fmt=".2f", cmap="Blues", cbar_kws={'label': 'V1 > Mediana (izq)'})
plt.title("Proporción V1 > Mediana por familia y outliers (recorte izquierda)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_left_v2_median, annot=True, fmt=".2f", cmap="Greens", cbar_kws={'label': 'V2 > Mediana (izq)'})
plt.title("Proporción V2 > Mediana por familia y outliers (recorte izquierda)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_left_v2_v1, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'V2 > V1 (izq)'})
plt.title("Proporción V2 > V1 por familia y outliers (recorte izquierda)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_right_v1_median, annot=True, fmt=".2f", cmap="Blues", cbar_kws={'label': 'V1 > Mediana (der)'})
plt.title("Proporción V1 > Mediana por familia y outliers (recorte derecha)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_right_v2_median, annot=True, fmt=".2f", cmap="Greens", cbar_kws={'label': 'V2 > Mediana (der)'})
plt.title("Proporción V2 > Mediana por familia y outliers (recorte derecha)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(heat_data_right_v2_v1, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'V2 > V1 (der)'})
plt.title("Proporción V2 > V1 por familia y outliers (recorte derecha)")
plt.ylabel("Familia")
plt.xlabel("Tuvo Outliers")
plt.show()


### **Resumen de significancia estadística**

A continuación se muestra un cuadro resumen de los resultados de los tests Wilcoxon (unilateral, alternativa 'greater') para cada comparación y tipo de recorte. Se considera que una métrica es estadísticamente superior si el p-valor < 0.05.


In [ ]:
# Cuadro resumen de significancia estadística

# Combinar los resultados de los tests
tests_all = pd.concat([
    df_tests_left.assign(recorte="izquierda"),
    df_tests_right.assign(recorte="derecha")
], ignore_index=True)

# Determinar si la métrica es estadísticamente superior (p < 0.05)
tests_all["significativo"] = tests_all["p_value"] < 0.05

# Añadir prueba V1 vs V2 (Wilcoxon unilateral, alternativa 'greater')
comparisons_left_v1_vs_v2 = [("d_v1_vs_v2_left", "V1 vs V2 (izq)")]
comparisons_right_v1_vs_v2 = [("d_v1_vs_v2_right", "V1 vs V2 (der)")]

df_tests_left_v1_vs_v2 = run_tests(df_ab, comparisons_left_v1_vs_v2, alternative="greater")
df_tests_right_v1_vs_v2 = run_tests(df_ab, comparisons_right_v1_vs_v2, alternative="greater")

# Unir los nuevos resultados
tests_all_v1_vs_v2 = pd.concat([
    df_tests_left_v1_vs_v2.assign(recorte="izquierda"),
    df_tests_right_v1_vs_v2.assign(recorte="derecha")
], ignore_index=True)
tests_all_v1_vs_v2["significativo"] = tests_all_v1_vs_v2["p_value"] < 0.05

# Concatenar ambos resultados
tests_all_total = pd.concat([tests_all, tests_all_v1_vs_v2], ignore_index=True)

# Cuadro pivote
cuadro = tests_all_total.pivot_table(
    index=["comparacion"],
    columns="recorte",
    values="significativo",
    aggfunc="first"
    )

# Mostrar el cuadro resumen
print("Cuadro resumen de significancia (True = superior estadísticamente, p < 0.05):")
display(cuadro)

## **Conclusiones de prueba de estabilidad por recortes**
Ambas métricas (V1 y V2) resultan significativamente más estables que el promedio ante recortes de colas por ambos lados en toda clase de distribuciones, no resultan más estables que la mediana debido a que la mediana es inherentemente estable ante recortes de datos, a menos que el recorte sea tan grande como para modificar el dato mediano. Por ello, la comparación de estabilidad propuesta no es ideal para comparar el desempeño de la métrica vs la mediana. 

# **Comparaciones métrica V1 vs. V2**

Se busca evidencia sobre un mejor desempeño de alguna de las métricas controlando por familias de distribuciones.

In [ ]:
# Diagnóstico de estabilidad V1 vs V2 por familia distributiva

# Crear un DataFrame para almacenar los resultados
resultados_familia = []

for fam, grupo in df_ab.groupby("familia"):
    # Diferencia de estabilidad V1 - V2 bajo recorte izquierdo
    dif_left = grupo["delta_mp1_left"] - grupo["delta_mp2_left"]
    stat_left, p_left = wilcoxon(dif_left, alternative="greater")
    
    # Diferencia de estabilidad V1 - V2 bajo recorte derecho
    dif_right = grupo["delta_mp1_right"] - grupo["delta_mp2_right"]
    stat_right, p_right = wilcoxon(dif_right, alternative="greater")
    
    resultados_familia.append({
        "familia": fam,
        "n": len(grupo),
        "mean_diff_left": dif_left.mean(),
        "wilcoxon_stat_left": stat_left,
        "p_value_left": p_left,
        "mean_diff_right": dif_right.mean(),
        "wilcoxon_stat_right": stat_right,
        "p_value_right": p_right
    })

df_diagnostico_familia = pd.DataFrame(resultados_familia)

# Mostrar resultados
print("Diagnóstico de estabilidad V1 vs V2 por familia distributiva")
display(df_diagnostico_familia)

# Opcional: marcar si V1 es significativamente más estable que V2 (p < 0.05)
df_diagnostico_familia["V1_mas_estable_izq"] = df_diagnostico_familia["p_value_left"] < 0.05
df_diagnostico_familia["V1_mas_estable_der"] = df_diagnostico_familia["p_value_right"] < 0.05

display(df_diagnostico_familia[["familia", "V1_mas_estable_izq", "V1_mas_estable_der"]])